In [ ]:
import numpy as np
from typing import Union, Callable
import pytest
import ipytest
from copy import deepcopy
np.random.seed(42)

## Fonctions d'activation

In [ ]:
def sigmoid(x: np.ndarray) -> np.ndarray:
    res = 1 / (1 + np.exp(- x))
    return res

def relu(x: np.ndarray) -> np.ndarray:
    x[x<0] = 0
    return x

def tanh(x: np.ndarray) -> np.ndarray:
    res = (np.exp(x) - np.exp(-x)) / (np.exp(x) + np.exp(-x))
    return res

## Propagation en avant

In [ ]:
def forward_propagation(inputs: np.ndarray,
               weights: list[np.ndarray],
               activation_functions: Union[Callable,list[Callable],list[list[Callable]]]) -> np.ndarray:
    """
    Returns the output of the network.

            Parameters:
                    inputs: is an array of shape n,p 
                    (with p being the number of input features and n the number of observations)
                    weights: list of matrices. 
                             Each matrix represents a layer. 
                             The number of rows is the number of nodes. 
                             The number of columns is the number inputs of the layer (with the bias)
                    activation_functions: the user can provide either
                        * one activation function for the whole network (one callable)
                        * one activation function for each layer (list of callable)
                        * one activation function per nodes (list of list of callable)

    """
    
    # number of nodes per layers
    network_structure = []
    for layer_weights in weights:
        network_structure.append(layer_weights.shape[0])

    # all activation functions
    activation_functions_parsed = parse_activation_function(activation_functions, network_structure)
    
    bias = np.repeat(np.array([[1]]), inputs.shape[0], axis=0)

    layer_input = np.append(inputs,bias,axis = 1).T

    for func,layer_weights in zip(activation_functions_parsed,weights):

        layer_combinaison = layer_weights.dot(layer_input)
        layer_output = apply_activation_functions(func,layer_combinaison)
        layer_input = np.append(layer_output,bias,axis = 1).T

    return layer_output

def parse_activation_function(activation_functions, network_structure) -> list[list[Callable]]:
    activation_functions_parsed = deepcopy(activation_functions)
    if isinstance(activation_functions, Callable):
        activation_functions_parsed = []
        for number_nodes in network_structure:
            layer_funcs = [activation_functions for n in range(0,number_nodes)]
            activation_functions_parsed.append(layer_funcs)
    
    if isinstance(activation_functions,list):
        if isinstance(activation_functions[0], Callable):
            activation_functions_parsed = []
            for func, number_nodes in zip(activation_functions, network_structure):
                activation_functions_parsed.append([func for n in range(0,number_nodes)])
    return activation_functions_parsed

def apply_activation_functions(functions: list[object], values: np.ndarray) -> np.ndarray:
    all_activated_values = np.zeros(values.shape)
    for i in range(0,values.shape[0]):
        row_values = values[i,:]
        all_activated_values[i,:] = [node_func(value) for node_func,value in zip(functions,row_values)]
    return np.array(all_activated_values).T

In [ ]:
def test_forward_nn_returns_correct_value():
    #given
    inputs = np.array([[1,2,5,4]])
    weights = [
        np.array(
            [
                [1,0.2,0.5,1,-1],
                [2,1,3,5,0],
                [0.2,0.1,0.6,0.78,1]
            ]
        )
    ]
    activation = sigmoid
    expected_output = np.array([[0.99899323, 1, 0.99945816]])
    #when
    output = forward_propagation(inputs,weights,activation)
    #then
    np.testing.assert_allclose(output, expected_output)
    
    
@pytest.mark.parametrize('activation', [
    (sigmoid),
    ([sigmoid]),
    ([[sigmoid,sigmoid,sigmoid]])
])
def test_forward_nn_returns_correct_value(activation):
    #given
    inputs = np.array([[1,2,5,4]])
    weights = [
        np.array(
            [
                [1,0.2,0.5,1,-1],
                [2,1,3,5,0],
                [0.2,0.1,0.6,0.78,1]
            ]
        )
    ]

    expected_output = np.array([[0.99899323, 1, 0.99945816]])
    #when
    output = forward_propagation(inputs,weights,activation)
    #then
    np.testing.assert_allclose(output, expected_output)

ipytest.run("-qq")

## Random weights

In [ ]:
def create_layer_weights(inputs_size: int,outputs_size: int)-> np.ndarray:
    avg = 0
    std = np.sqrt(1/inputs_size)
    layer_weights = np.random.normal(avg,std,(outputs_size,inputs_size))
    
    return layer_weights

def create_weights(network_structure: list[int]) -> list[np.ndarray]:
    np.random.seed(42)
    weights = []
    for layer in range(0,len(network_structure)-1):
        weights.append(create_layer_weights(
            inputs_size = network_structure[layer] + 1,
            outputs_size = network_structure[layer + 1]
        ))
    return weights

## vector to weights and back

In [ ]:
def vector_to_weights(vector: np.ndarray,network_structure: list[int]) -> list[np.ndarray]:
    weights = []
    for layer in range(0,len(network_structure)-1):
        input_size = (network_structure[layer] + 1)
        output_size = network_structure[layer + 1]
        selected_parameters = np.array(vector[:(input_size * output_size)])
        weights.append(selected_parameters.reshape((output_size,input_size)))
        vector = vector[(input_size * output_size):]
    
    return weights

def weights_to_vector(weights: list[np.ndarray]):
    vector = []
    network_structure = []
    for layer_weight in weights:
        vector = vector + layer_weight.reshape((1,-1))[0].tolist()
        if len(network_structure) == 0:
            network_structure = network_structure + [layer_weight.shape[1] - 1] 
        network_structure = network_structure + [layer_weight.shape[0]] 
        
    return np.array(vector), network_structure

In [ ]:
def test_vector_to_weights_returns_correct_value():
    # given
    vector = [0.2027827,-0.05644616,0.26441774,0.62177434,-0.09559271,-0.09558601,
              0.64471093,  0.31330392, -0.19166212,  0.22149921, -0.18918948,-0.19013338,
              0.09878068, -0.78109339, -0.70419476, -0.22955292, -0.41348657,0.12829094,
              -0.45401204, -0.70615185,  0.73282438, -0.11288815]
    network_structure = [5,3,1]
    expected_weights = [np.array([[ 0.2027827 , -0.05644616,  0.26441774,  0.62177434, -0.09559271,-0.09558601],
                               [ 0.64471093,  0.31330392, -0.19166212,  0.22149921, -0.18918948,-0.19013338],
                               [ 0.09878068, -0.78109339, -0.70419476, -0.22955292, -0.41348657,0.12829094]]),
                        np.array([[-0.45401204, -0.70615185,  0.73282438, -0.11288815]])]
    # when
    weights = vector_to_weights(vector,network_structure)
    # then
    for layer_weight,expected_layer_weight in zip(weights,expected_weights):
        np.testing.assert_allclose(layer_weight, expected_layer_weight)

def test_weights_to_vector_returns_correct_value():
    # given
    weights = [np.array([[ 0.2027827 , -0.05644616,  0.26441774,  0.62177434, -0.09559271,-0.09558601],
                               [ 0.64471093,  0.31330392, -0.19166212,  0.22149921, -0.18918948,-0.19013338],
                               [ 0.09878068, -0.78109339, -0.70419476, -0.22955292, -0.41348657,0.12829094]]),
                        np.array([[-0.45401204, -0.70615185,  0.73282438, -0.11288815]])]
    expected_vector = np.array([0.2027827,-0.05644616,0.26441774,0.62177434,-0.09559271,-0.09558601,
              0.64471093,  0.31330392, -0.19166212,  0.22149921, -0.18918948,-0.19013338,
              0.09878068, -0.78109339, -0.70419476, -0.22955292, -0.41348657,0.12829094,
              -0.45401204, -0.70615185,  0.73282438, -0.11288815])
    expected_structure = [5,3,1]
    # when
    vector, network_structure = weights_to_vector(weights)
    
    np.testing.assert_allclose(vector, expected_vector)
    np.testing.assert_allclose(network_structure, expected_structure)

ipytest.run("-qq")

## Multiple inputs 

In [ ]:
inputs = np.array([[1,2,5,4]])
weights = [
        np.array(
            [
                [1,0.2,0.5,1,-1],
                [2,1,3,5,0],
                [0.2,0.1,0.6,0.78,1]
            ]
        )
    ]
activation = sigmoid
forward_propagation(inputs,weights,activation)

In [ ]:
inputs = np.array([[1,0.2,0.15,0.024]])
weights = [
        np.array(
            [
                [1,0.2,0.5,1,-1],
                [2,1,3,5,0],
                [0.2,0.1,0.6,0.78,1]
            ]
        )
    ]
activation = sigmoid
forward_propagation(inputs,weights,activation)

In [ ]:
inputs = np.array([[1,2,5,4],[1,0.2,0.15,0.024]])
weights = [
        np.array(
            [
                [1,0.2,0.5,1,-1],
                [2,1,3,5,0],
                [0.2,0.1,0.6,0.78,1]
            ]
        )
    ]
activation = sigmoid
forward_propagation(inputs,weights,activation)

In [ ]:
inputs = np.array([[1,2,5,4],[1,0.2,0.15,0.024]])
weights = [
        np.array(
            [
                [1,0.2,0.5,1,-1],
                [2,1,3,5,0],
                [0.2,0.1,0.6,0.78,1]
            ]
        ),
    np.array(
            [
                [1,0.2,0.5,1],
                [2,1,3,5]
            ]
        )
    ]
activation = sigmoid
forward_propagation(inputs,weights,activation)

### Creating a data set 
Create a data set of points sampled randomly from a function.

In [4]:
import test_functions
import numpy as np
fun = test_functions.sphere
dim = 2
LB = [-5] * dim
UB = [5] * dim
ndata = 10

entry_data = np.random.uniform(low=LB,high=UB,size=(ndata,dim))
entry_data
ydata = fun(xdata)

array([[ 4.78129114, -1.5960368 ],
       [-4.05969221, -0.70229707],
       [-1.69742218,  2.27013424],
       [-4.30673085, -4.95803344],
       [-0.71706254,  3.43746721],
       [ 3.96397923,  4.41931501],
       [-2.28613263, -1.32562427],
       [ 2.34265092,  2.45378602],
       [-1.80707864,  2.27599077],
       [ 2.6214833 ,  3.41603093]])